In [1]:
import torch
import torch.nn as nn

class DynamicLinear(nn.Module):
    def __init__(self, input_features, output_features):
        super(DynamicLinear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features
        
        # 动态创建权重和偏置参数，但不使用内置的 nn.Linear
        weight = torch.randn(output_features, input_features)
        bias = torch.randn(output_features)
        
        # 注册参数
        self.register_parameter('weight', nn.Parameter(weight))
        self.register_parameter('bias', nn.Parameter(bias))
    
    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

# 使用自定义的 DynamicLinear
model = nn.Sequential(
    DynamicLinear(10, 5),
    nn.ReLU(),
    DynamicLinear(5, 1)
)

# 打印模型的参数
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, Parameter size: {param.size()}")


Parameter name: 0.weight, Parameter size: torch.Size([5, 10])
Parameter name: 0.bias, Parameter size: torch.Size([5])
Parameter name: 2.weight, Parameter size: torch.Size([1, 5])
Parameter name: 2.bias, Parameter size: torch.Size([1])


In [3]:
class OptionalBias(nn.Module):
    def __init__(self, input_features, output_features, use_bias=True):
        super(OptionalBias, self).__init__()
        self.weight = nn.Parameter(torch.randn(output_features, input_features))
        
        if use_bias:
            bias = torch.randn(output_features)
            self.register_parameter('bias', nn.Parameter(bias))
        else:
            # 如果不使用偏置，则将 bias 设置为 None
            self.register_parameter('bias', None)
    
    def forward(self, x):
        return torch.nn.functional.linear(x, self.weight, self.bias)

# 实例化不使用偏置的层
layer_no_bias = OptionalBias(10, 5, use_bias=False)

# 实例化使用偏置的层
layer_with_bias = OptionalBias(10, 5, use_bias=True)

# 检查参数
print("Without bias:")
for name, param in layer_no_bias.named_parameters():
    print(f"Parameter name: {name}, Parameter size: {param.size() if param is not None else 'None'}")

print("\nWith bias:")
for name, param in layer_with_bias.named_parameters():
    print(f"Parameter name: {name}, Parameter size: {param.size()}")


Without bias:
Parameter name: weight, Parameter size: torch.Size([5, 10])

With bias:
Parameter name: weight, Parameter size: torch.Size([5, 10])
Parameter name: bias, Parameter size: torch.Size([5])


In [2]:
class ExternalParameterModule(nn.Module):
    def __init__(self, parameter_dict):
        super(ExternalParameterModule, self).__init__()
        for name, value in parameter_dict.items():
            param = nn.Parameter(value)
            self.register_parameter(name, param)
    
    def forward(self, x):
        # 假设我们使用这些参数进行一些计算
        for name, param in self.named_parameters():
            x = x * param
        return x

# 从外部加载参数值
external_params = {
    'scale_factor': torch.tensor(2.0),
    'shift_value': torch.tensor(1.0)
}

# 创建模型实例
model = ExternalParameterModule(external_params)

# 查看模型的参数
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, Parameter value: {param.item()}")

# 使用模型进行计算
input_data = torch.tensor([3.0])
output = model(input_data)
print(f"Output: {output.item()}")


Parameter name: scale_factor, Parameter value: 2.0
Parameter name: shift_value, Parameter value: 1.0
Output: 6.0
